# Rozpoznawanie gatunków muzycznych za pomocą uczenia maszynowego
## Tworzenie modelu
#### Bartosz Dybowski

### 1. Wyodrębnienie cech z utworów i stworzenie tabeli z danymi

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import librosa

In [2]:
# Ścieżka do folderu z plikami WAV
folder_path = "zbiór utworów"

# Listy przechowujące cechy i przypisane gatunki
data = []
genres = []

# Iteracja po plikach w folderze
for file_name in os.listdir(folder_path):
    if file_name.endswith('.wav'):
        file_path = os.path.join(folder_path, file_name)
        y, sr = librosa.load(file_path)
        onset_env = librosa.onset.onset_strength(y=y, sr=sr)
        
        # Wyodrębnienie cech
        tempo = librosa.feature.tempo(onset_envelope=onset_env, sr=sr)  # Tempo 
        key, _ = librosa.beat.beat_track(y=y, sr=sr)  # Tonacja
        zero_crossing_rate = librosa.feature.zero_crossing_rate(y).mean()  # Liczba przejść przez zero
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr).mean()  # Środek ciężkości widma
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr).mean()  # Szerokość pasm spektralnych
        spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr).mean()  # Kontrast spektralny
        spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr).mean()  # Rolloff spektralny
        spectral_flatness = librosa.feature.spectral_flatness(y=y).mean()  # Płaskość spektralna
        rms = librosa.feature.rms(y=y).mean()  # Skuteczna wartość średnia (RMS - Root Mean Square)
        harmonic_to_noise = librosa.effects.harmonic(y).mean()  # Stosunek harmoniczności do szumu(Harmonic-to-Noise Ratio)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr).mean()  # Chromagram
        chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr).mean()
        mfcc = librosa.feature.mfcc(y=y, sr=sr).mean()  # Współczynniki cepstralne (MFCCs)
        pitches, magnitudes = librosa.piptrack(y=y, sr=sr)  # Częstotliwość fundamentalna (Pitch)
        rhythmics = librosa.feature.tempogram(y=y, sr=sr).mean()  # Rytmika

        # Dodanie cech do listy danych
        data.append([tempo, key, zero_crossing_rate, spectral_centroid, spectral_bandwidth, 
                     spectral_contrast, spectral_rolloff, spectral_flatness, rms, harmonic_to_noise, 
                     chroma_stft, chroma_cens, mfcc, pitches.mean(), rhythmics])
        
        # Odczyt gatunku z nazwy pliku (nazwa pliku jest w formacie "nazwaGatunku_numer.wav")
        genre = file_name.split('_')[0]
        genres.append(genre)

# Utworzenie ramki danych Pandas
df = pd.DataFrame(data, columns=['tempo', 'key', 'zero_crossing_rate', 'spectral_centroid', 'spectral_bandwidth', 
                                 'spectral_contrast', 'spectral_rolloff', 'spectral_flatness', 'rms', 'harmonic_to_noise', 
                                 'chroma_stft', 'chroma_cens', 'mfcc', 'pitches', 'rhythmics'])

# Dodanie kolumny z gatunkiem
df['genre'] = genres  

In [3]:
# Przekształcenie kolumny tempo na typ danych numeryczny
df['tempo'] = df['tempo'].astype(float)

In [4]:
#Tabela z danymi
df

### 2. Analiza i wybór cech (atrybutów)

In [5]:
# Tabela z samymi atrybutami (bez przypisanych klas)
attributes_df = df.drop(columns=['genre'])
attributes_df

In [6]:
sns.pairplot(df, kind = "scatter", hue = "genre")

In [7]:
# Analiza korelacji
correlation_matrix = attributes_df.corr()
plt.figure(figsize=(12,9), dpi = 100)
sns.heatmap(correlation_matrix, annot=True)

Po analizie i dokonaniu wyboru atrybutów

In [147]:
df_reduced = df.drop(columns=['spectral_contrast', 'chroma_cens', 'rhythmics', 'spectral_bandwidth'])

### 3. Klasyfikacja

In [148]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.datasets import make_classification

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [149]:
def divide(dataframe, proportion):
    dscr_lrn, dscr_test, dec_lrn, dec_test = train_test_split(dataframe.iloc[:, 0:-1], 
                                            dataframe.iloc[:, -1].astype('category').cat.codes, test_size = proportion, 
                                            stratify = dataframe.iloc[:, -1].astype('category').cat.codes, random_state=42)
    return {'dscr_lrn':dscr_lrn, 'dscr_test':dscr_test, 'dec_lrn':dec_lrn, 'dec_test':dec_test}

def verify(model, dscr_lrn, dscr_test, dec_lrn, dec_test, attrib):
    model.fit(dscr_lrn.iloc[:, attrib], dec_lrn)
    lrn_res = model.predict(dscr_lrn.iloc[:, attrib])
    test_res = model.predict(dscr_test.iloc[:, attrib])
    
    mp = confusion_matrix(dec_lrn, lrn_res)
    print('Macierz pomylek dla zbioru uczącego, dokładność:',np.sum(np.diag(mp))/np.sum(mp))
    print(pd.crosstab(dec_lrn, lrn_res))
    
    mp = confusion_matrix(dec_test, test_res)
    print('Macierz pomylek dla zbioru testowego, dokładność:',np.sum(np.diag(mp))/np.sum(mp))
    print(pd.crosstab(dec_test, test_res))

def lims(model, dscr_lrn, dscr_test, dec_lrn, dec_test, x, y, contour = 1):
    if (contour == 1):    
        model.fit(dscr_lrn.iloc[:, [x, y]], dec_lrn)
        x_min = min(dscr_lrn.iloc[:, x].min(), dscr_test.iloc[:, x].min())
        x_max = max(dscr_lrn.iloc[:, x].max(), dscr_test.iloc[:, x].max())
        y_min = min(dscr_lrn.iloc[:, y].min(), dscr_test.iloc[:, y].min())
        y_max = max(dscr_lrn.iloc[:, y].max(), dscr_test.iloc[:, y].max())
        range_x = x_max - x_min
        range_y = y_max - y_min
        x_min = x_min - 0.1 * range_x
        x_max = x_max + 0.1 * range_x
        y_min = y_min - 0.1 * range_y
        y_max = y_max + 0.1 * range_y       
        xx, yy = np.meshgrid(np.arange(x_min, x_max, (x_max - x_min) / 150), 
                    np.arange(y_min, y_max, (y_max - y_min) / 150))
        Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)
    plt.figure(dpi = 100)
    if (contour == 1):
        plt.contourf(xx, yy, Z, levels = 4, alpha = 0.2)
    plt.scatter(dscr_lrn.iloc[:, x], dscr_lrn.iloc[:, y], c = dec_lrn, marker = '.')
    plt.scatter(dscr_test.iloc[:, x], dscr_test.iloc[:, y], c = dec_test, marker = 'x') 

bez analizy atrybutów

In [150]:
# Podział zbioru na uczący i testowy w proporcji 70:30
df_lrn_and_test1 = divide(df, 0.3)
df_dscr_lrn1 = df_lrn_and_test1['dscr_lrn']
df_dscr_test1 = df_lrn_and_test1['dscr_test']
df_dec_lrn1 = df_lrn_and_test1['dec_lrn']
df_dec_test1 = df_lrn_and_test1['dec_test']

print('Liczba obiektów zbioru uczącego: ', len(df_dscr_lrn1))
print('Liczba obiektów zbioru testowego: ', len(df_dscr_test1))

In [151]:
#Lista wszystkich atrybutów
atr_list1 = list(range(df.shape[1]-1))

z analizą atrybutów

In [152]:
# Podział zbioru na uczący i testowy w proporcji 70:30
df_lrn_and_test2 = divide(df_reduced, 0.3)
df_dscr_lrn2 = df_lrn_and_test2['dscr_lrn']
df_dscr_test2 = df_lrn_and_test2['dscr_test']
df_dec_lrn2 = df_lrn_and_test2['dec_lrn']
df_dec_test2 = df_lrn_and_test2['dec_test']

print('Liczba obiektów zbioru uczącego: ', len(df_dscr_lrn2))
print('Liczba obiektów zbioru testowego: ', len(df_dscr_test2))

In [153]:
#Lista wszystkich atrybutów
atr_list2 = list(range(df_reduced.shape[1]-1))

#### Naiwny klasyfikator Bayesa 

In [154]:
# Tworzenie modelu naiwnego klasyfikatora Bayesa (wszystkie cechy)
bayes1 = GaussianNB()

verify(bayes1, df_dscr_lrn1, df_dscr_test1, df_dec_lrn1, df_dec_test1, atr_list1)
lims(bayes1, df_dscr_lrn1, df_dscr_test1, df_dec_lrn1, df_dec_test1, 0, 1)

# Trenowanie modelu
bayes1.fit(df_dscr_lrn1, df_dec_lrn1)

# Ocena wydajności modelu
y_pred = bayes1.predict(df_dscr_test1)
accuracy = accuracy_score(df_dec_test1, y_pred)
print("Dokładność modelu:", accuracy)
print("Raport klasyfikacji:")
print(classification_report(df_dec_test1, y_pred))

In [155]:
# Tworzenie modelu naiwnego klasyfikatora Bayesa (wybrane cechy)
bayes2 = GaussianNB()

verify(bayes2, df_dscr_lrn2, df_dscr_test2, df_dec_lrn2, df_dec_test2, atr_list2)
lims(bayes2, df_dscr_lrn2, df_dscr_test2, df_dec_lrn2, df_dec_test2, 0, 1)

# Trenowanie modelu
bayes2.fit(df_dscr_lrn2, df_dec_lrn2)

# Ocena wydajności modelu
y_pred = bayes2.predict(df_dscr_test2)
accuracy = accuracy_score(df_dec_test2, y_pred)
print("Dokładność modelu:", accuracy)
print("Raport klasyfikacji:")
print(classification_report(df_dec_test2, y_pred))

#### Las losowy

In [156]:
# Tworzenie modelu lasu losowego (wszystkie cechy)
randomForest1 = RandomForestClassifier(n_estimators=100, random_state=42)

verify(randomForest1, df_dscr_lrn1, df_dscr_test1, df_dec_lrn1, df_dec_test1, atr_list1)
lims(randomForest1, df_dscr_lrn1, df_dscr_test1, df_dec_lrn1, df_dec_test1, 0, 1)

# Trenowanie modelu
randomForest1.fit(df_dscr_lrn1, df_dec_lrn1)

# Ocena wydajności modelu
y_pred = randomForest1.predict(df_dscr_test1)
accuracy = accuracy_score(df_dec_test1, y_pred)
print("Dokładność modelu:", accuracy)
print("Raport klasyfikacji:")
print(classification_report(df_dec_test1, y_pred))

In [161]:
# Tworzenie modelu lasu losowego (wybrane cechy)
randomForest2 = RandomForestClassifier(n_estimators=100, random_state=42)

verify(randomForest2, df_dscr_lrn2, df_dscr_test2, df_dec_lrn2, df_dec_test2, atr_list2)
lims(randomForest2, df_dscr_lrn2, df_dscr_test2, df_dec_lrn2, df_dec_test2, 0, 1)

# Trenowanie modelu
randomForest2.fit(df_dscr_lrn2, df_dec_lrn2)

# Ocena wydajności modelu
y_pred = randomForest2.predict(df_dscr_test2)
accuracy = accuracy_score(df_dec_test2, y_pred)
print("Dokładność modelu:", accuracy)
print("Raport klasyfikacji:")
print(classification_report(df_dec_test2, y_pred))

#### XGBoost

In [158]:
# Tworzenie modelu XGBoost (wszystkie cechy)
xgb_model1 = XGBClassifier()

verify(xgb_model1, df_dscr_lrn1, df_dscr_test1, df_dec_lrn1, df_dec_test1, atr_list1)
lims(xgb_model1, df_dscr_lrn1, df_dscr_test1, df_dec_lrn1, df_dec_test1, 0, 1)

# Trenowanie modelu
xgb_model1.fit(df_dscr_lrn1, df_dec_lrn1)

# Ocena wydajności modelu
y_pred = xgb_model1.predict(df_dscr_test1)
accuracy = accuracy_score(df_dec_test1, y_pred)
print("Dokładność modelu:", accuracy)
print("Raport klasyfikacji:")
print(classification_report(df_dec_test1, y_pred))

In [159]:
# Tworzenie modelu XGBoost (wybrane cechy)
xgb_model2 = XGBClassifier()

verify(xgb_model2, df_dscr_lrn2, df_dscr_test2, df_dec_lrn2, df_dec_test2, atr_list2)
lims(xgb_model2, df_dscr_lrn2, df_dscr_test2, df_dec_lrn2, df_dec_test2, 0, 1)

# Trenowanie modelu
xgb_model2.fit(df_dscr_lrn2, df_dec_lrn2)

# Ocena wydajności modelu
y_pred = xgb_model2.predict(df_dscr_test2)
accuracy = accuracy_score(df_dec_test2, y_pred)
print("Dokładność modelu:", accuracy)
print("Raport klasyfikacji:")
print(classification_report(df_dec_test2, y_pred))

Po przeanalizowaniu wszystkich wyników klasyfikacji, wybieram Las Losowy dla zredukowanych danych.

In [162]:
# Zapisanie modelu
import pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(randomForest2, file)